# [Sec 0] submit a avg score csv
data set in trainging.csv
- id: ad identifier
- click: 0/1 for non-click/click
- hour: format is YYMMDDHH, so 14091123 means 23:00 on Sept. 11, 2014 UTC. *
- C1 – anonymized categorical variable *
- banner_pos
- site_id *
- site_domain *
- site_category *
- app_id *
- app_domain *
- app_category  *
- device_id
- device_ip *
- device_model
- device_type *
- device_conn_type *
- C14-C21 – anonymized categorical variables

In [ ]:
import pandas as pd

In [ ]:
sampleSubmission = pd.read_csv("data/sampleSubmission")

In [ ]:
sampleSubmission.click = 0.16980562476404604
sampleSubmission.head()

In [ ]:
# all avg-score can reach rank (1314/1604) 81 %
sampleSubmission.to_csv("avg_submission.csv",index=False)

# [Sec1] Train a xgb model with minimum features
```
pip install xgboost
```

In [ ]:
import random

In [ ]:
# creating cv set.
import pickle
N= 40428967 
s = int(N*3/10) #desired sample size : 30%

random.seed(5566)
skip = sorted(random.sample(range(1,N+1),N-s)) 
with open('data/cv_5566', 'wb') as fp:
    pickle.dump(skip, fp)
    
random.seed(5567)
skip = sorted(random.sample(range(1,N+1),N-s)) 
with open('data/cv_5567', 'wb') as fp:
    pickle.dump(skip, fp)
    
random.seed(5568)
skip = sorted(random.sample(range(1,N+1),N-s)) 
with open('data/cv_5568', 'wb') as fp:
    pickle.dump(skip, fp)
    
random.seed(5569)
skip = sorted(random.sample(range(1,N+1),N-s)) 
with open('data/cv_5569', 'wb') as fp:
    pickle.dump(skip, fp)
    
random.seed(5570)
skip = sorted(random.sample(range(1,N+1),N-s)) 
with open('data/cv_5570', 'wb') as fp:
    pickle.dump(skip, fp)

In [1]:
import pandas as pd
import numpy as np
import random
import pickle
#===fold===
fold=5570
#===fold===
# training data is to large, sample ~0.5N
with open ('./data/cv_'+str(fold), 'rb') as fp:
    skip = pickle.load(fp)
X_train = pd.read_csv("data/train", skiprows=skip)


In [2]:
# feat
from sklearn.feature_extraction import FeatureHasher
hasher = FeatureHasher(n_features=10,
            non_negative=True,
            input_type='string')

# create features

## site
tmp_features = hasher.transform(X_train.site_id.tolist()).toarray()
for i in range(10):
    X_train['site_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_train.site_domain.tolist()).toarray()
for i in range(10):
    X_train['site_domain'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_train.site_category.tolist()).toarray()
for i in range(10):
    X_train['site_category'+str(i)]=tmp_features[:,i]

## app
tmp_features = hasher.transform(X_train.app_id.tolist()).toarray()
for i in range(10):
    X_train['app_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_train.app_domain.tolist()).toarray()
for i in range(10):
    X_train['app_domain'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_train.app_category.tolist()).toarray()
for i in range(10):
    X_train['app_category'+str(i)]=tmp_features[:,i]

## devices
tmp_features = hasher.transform(X_train.device_id.tolist()).toarray()
for i in range(10):
    X_train['device_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_train.device_ip.tolist()).toarray()
for i in range(10):
    X_train['device_ip'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_train.device_model.tolist()).toarray()
for i in range(10):
    X_train['device_model'+str(i)]=tmp_features[:,i]

In [3]:
Site_list = ['site_id'+str(i) for i in range(10)]+['site_domain'+str(i) for i in range(10)]+ \
            ['site_category'+str(i) for i in range(10)]

App_list = ['app_id'+str(i) for i in range(10)]+['app_domain'+str(i) for i in range(10)]+ \
            ['app_category'+str(i) for i in range(10)]

Devices_list = ['device_id'+str(i) for i in range(10)]+['device_ip'+str(i) for i in range(10)]+ \
            ['device_model'+str(i) for i in range(10)]
X_train_trial = X_train[['banner_pos','C1','device_type','device_conn_type','C14','C15','C16','C17','C18',\
                         'C19','C20','C21']+Site_list+App_list+Devices_list]
y_target = X_train.click.values

In [4]:
X_train_trial.shape

(12128690, 102)

In [5]:
skip=[]
X_train=[]
import gc
gc.collect()

171

## [Sec1-1] train with xgboost

In [ ]:
import xgboost as xgb

# make xgb dataset
random.seed(5566)
msk = np.random.rand(len(X_train_trial)) < 0.8
dtrain = xgb.DMatrix(X_train_trial[msk], y_target[msk])
dtest = xgb.DMatrix(X_train_trial[~msk], y_target[~msk])

In [ ]:
param = {'max_depth':8, 'eta':0.5, 'silent':1, 'objective':'binary:logistic' ,'subsample':0.3}
param['nthread'] = 6
param['eval_metric'] = 'logloss'
evallist  = [ (dtrain,'train'),(dtest,'eval')]
num_round = 8000
#bst = xgb.train( param, dtrain, num_round, evallist , early_stopping_rounds=300)

## [Sec1-2] train with lightgbm
xgboost is too slowly and memory require much.

In [6]:
import lightgbm as lgb

In [7]:
#create lightgbm dataset
msk = np.random.rand(len(X_train_trial)) < 0.8
lgb_train = lgb.Dataset(X_train_trial[msk], y_target[msk])
lgb_eval = lgb.Dataset(X_train_trial[~msk], y_target[~msk],reference=lgb_train)


In [8]:
X_train_trial=[]
import gc
gc.collect()

0

In [9]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': { 'binary_logloss'},
    'num_leaves': 31, # defauly leaves(31) amount for each tree
    'learning_rate': 0.08,
    'feature_fraction': 0.7, # will select 70% features before training each tree
    'bagging_fraction': 0.3, #feature_fraction, but this will random select part of data
    'bagging_freq': 5, #  perform bagging at every 5 iteration
    'verbose': 0
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=4000,
                valid_sets=lgb_eval,
                early_stopping_rounds=500)

Start training...
[1]	valid_0's binary_logloss: 0.656543
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's binary_logloss: 0.625346
[3]	valid_0's binary_logloss: 0.598641
[4]	valid_0's binary_logloss: 0.575655
[5]	valid_0's binary_logloss: 0.555876
[6]	valid_0's binary_logloss: 0.538639
[7]	valid_0's binary_logloss: 0.523691
[8]	valid_0's binary_logloss: 0.510586
[9]	valid_0's binary_logloss: 0.498969
[10]	valid_0's binary_logloss: 0.488855
[11]	valid_0's binary_logloss: 0.479951
[12]	valid_0's binary_logloss: 0.472144
[13]	valid_0's binary_logloss: 0.465263
[14]	valid_0's binary_logloss: 0.459276
[15]	valid_0's binary_logloss: 0.453866
[16]	valid_0's binary_logloss: 0.449073
[17]	valid_0's binary_logloss: 0.444824
[18]	valid_0's binary_logloss: 0.44109
[19]	valid_0's binary_logloss: 0.43783
[20]	valid_0's binary_logloss: 0.434947
[21]	valid_0's binary_logloss: 0.432363
[22]	valid_0's binary_logloss: 0.429981
[23]	valid_0's binary_logloss: 0.427975
[24]	valid

[202]	valid_0's binary_logloss: 0.401022
[203]	valid_0's binary_logloss: 0.401011
[204]	valid_0's binary_logloss: 0.400988
[205]	valid_0's binary_logloss: 0.400956
[206]	valid_0's binary_logloss: 0.400933
[207]	valid_0's binary_logloss: 0.400906
[208]	valid_0's binary_logloss: 0.400894
[209]	valid_0's binary_logloss: 0.400864
[210]	valid_0's binary_logloss: 0.400838
[211]	valid_0's binary_logloss: 0.400824
[212]	valid_0's binary_logloss: 0.400809
[213]	valid_0's binary_logloss: 0.400791
[214]	valid_0's binary_logloss: 0.40078
[215]	valid_0's binary_logloss: 0.400758
[216]	valid_0's binary_logloss: 0.400725
[217]	valid_0's binary_logloss: 0.400707
[218]	valid_0's binary_logloss: 0.400671
[219]	valid_0's binary_logloss: 0.400649
[220]	valid_0's binary_logloss: 0.400623
[221]	valid_0's binary_logloss: 0.400595
[222]	valid_0's binary_logloss: 0.400564
[223]	valid_0's binary_logloss: 0.400541
[224]	valid_0's binary_logloss: 0.400517
[225]	valid_0's binary_logloss: 0.400506
[226]	valid_0's b

[403]	valid_0's binary_logloss: 0.398173
[404]	valid_0's binary_logloss: 0.398169
[405]	valid_0's binary_logloss: 0.398161
[406]	valid_0's binary_logloss: 0.398149
[407]	valid_0's binary_logloss: 0.398138
[408]	valid_0's binary_logloss: 0.398128
[409]	valid_0's binary_logloss: 0.398121
[410]	valid_0's binary_logloss: 0.398114
[411]	valid_0's binary_logloss: 0.398111
[412]	valid_0's binary_logloss: 0.398106
[413]	valid_0's binary_logloss: 0.398102
[414]	valid_0's binary_logloss: 0.398099
[415]	valid_0's binary_logloss: 0.398095
[416]	valid_0's binary_logloss: 0.39809
[417]	valid_0's binary_logloss: 0.398081
[418]	valid_0's binary_logloss: 0.398072
[419]	valid_0's binary_logloss: 0.398063
[420]	valid_0's binary_logloss: 0.398054
[421]	valid_0's binary_logloss: 0.398045
[422]	valid_0's binary_logloss: 0.398042
[423]	valid_0's binary_logloss: 0.398036
[424]	valid_0's binary_logloss: 0.398024
[425]	valid_0's binary_logloss: 0.398012
[426]	valid_0's binary_logloss: 0.397997
[427]	valid_0's b

[604]	valid_0's binary_logloss: 0.396831
[605]	valid_0's binary_logloss: 0.396828
[606]	valid_0's binary_logloss: 0.396817
[607]	valid_0's binary_logloss: 0.396811
[608]	valid_0's binary_logloss: 0.39681
[609]	valid_0's binary_logloss: 0.396808
[610]	valid_0's binary_logloss: 0.396805
[611]	valid_0's binary_logloss: 0.396797
[612]	valid_0's binary_logloss: 0.39679
[613]	valid_0's binary_logloss: 0.396784
[614]	valid_0's binary_logloss: 0.396779
[615]	valid_0's binary_logloss: 0.396772
[616]	valid_0's binary_logloss: 0.39677
[617]	valid_0's binary_logloss: 0.396769
[618]	valid_0's binary_logloss: 0.396768
[619]	valid_0's binary_logloss: 0.396767
[620]	valid_0's binary_logloss: 0.39676
[621]	valid_0's binary_logloss: 0.396757
[622]	valid_0's binary_logloss: 0.396756
[623]	valid_0's binary_logloss: 0.396755
[624]	valid_0's binary_logloss: 0.396753
[625]	valid_0's binary_logloss: 0.396751
[626]	valid_0's binary_logloss: 0.396747
[627]	valid_0's binary_logloss: 0.396745
[628]	valid_0's bina

[805]	valid_0's binary_logloss: 0.39604
[806]	valid_0's binary_logloss: 0.396039
[807]	valid_0's binary_logloss: 0.396037
[808]	valid_0's binary_logloss: 0.396032
[809]	valid_0's binary_logloss: 0.396032
[810]	valid_0's binary_logloss: 0.396027
[811]	valid_0's binary_logloss: 0.396023
[812]	valid_0's binary_logloss: 0.396021
[813]	valid_0's binary_logloss: 0.396019
[814]	valid_0's binary_logloss: 0.396018
[815]	valid_0's binary_logloss: 0.396017
[816]	valid_0's binary_logloss: 0.396014
[817]	valid_0's binary_logloss: 0.396008
[818]	valid_0's binary_logloss: 0.395998
[819]	valid_0's binary_logloss: 0.395994
[820]	valid_0's binary_logloss: 0.395992
[821]	valid_0's binary_logloss: 0.395987
[822]	valid_0's binary_logloss: 0.395986
[823]	valid_0's binary_logloss: 0.395981
[824]	valid_0's binary_logloss: 0.39598
[825]	valid_0's binary_logloss: 0.395979
[826]	valid_0's binary_logloss: 0.395974
[827]	valid_0's binary_logloss: 0.395968
[828]	valid_0's binary_logloss: 0.395965
[829]	valid_0's bi

...

[3163]	valid_0's binary_logloss: 0.393366
[3164]	valid_0's binary_logloss: 0.393366
[3165]	valid_0's binary_logloss: 0.393366
[3166]	valid_0's binary_logloss: 0.393366
[3167]	valid_0's binary_logloss: 0.393365
[3168]	valid_0's binary_logloss: 0.393365
[3169]	valid_0's binary_logloss: 0.393364
[3170]	valid_0's binary_logloss: 0.393363
[3171]	valid_0's binary_logloss: 0.393363
[3172]	valid_0's binary_logloss: 0.393363
[3173]	valid_0's binary_logloss: 0.393362
[3174]	valid_0's binary_logloss: 0.393361
[3175]	valid_0's binary_logloss: 0.393361
[3176]	valid_0's binary_logloss: 0.39336
[3177]	valid_0's binary_logloss: 0.39336
[3178]	valid_0's binary_logloss: 0.393359
[3179]	valid_0's binary_logloss: 0.39336
[3180]	valid_0's binary_logloss: 0.39336
[3181]	valid_0's binary_logloss: 0.393361
[3182]	valid_0's binary_logloss: 0.39336
[3183]	valid_0's binary_logloss: 0.39336
[3184]	valid_0's binary_logloss: 0.39336
[3185]	valid_0's binary_logloss: 0.39336
[3186]	valid_0's binary_logloss: 0.39336
[

[3359]	valid_0's binary_logloss: 0.393284
[3360]	valid_0's binary_logloss: 0.393284
[3361]	valid_0's binary_logloss: 0.393284
[3362]	valid_0's binary_logloss: 0.393285
[3363]	valid_0's binary_logloss: 0.393284
[3364]	valid_0's binary_logloss: 0.393283
[3365]	valid_0's binary_logloss: 0.393282
[3366]	valid_0's binary_logloss: 0.393283
[3367]	valid_0's binary_logloss: 0.393283
[3368]	valid_0's binary_logloss: 0.393283
[3369]	valid_0's binary_logloss: 0.393282
[3370]	valid_0's binary_logloss: 0.393282
[3371]	valid_0's binary_logloss: 0.393281
[3372]	valid_0's binary_logloss: 0.393282
[3373]	valid_0's binary_logloss: 0.393281
[3374]	valid_0's binary_logloss: 0.39328
[3375]	valid_0's binary_logloss: 0.39328
[3376]	valid_0's binary_logloss: 0.393279
[3377]	valid_0's binary_logloss: 0.393277
[3378]	valid_0's binary_logloss: 0.393278
[3379]	valid_0's binary_logloss: 0.393277
[3380]	valid_0's binary_logloss: 0.393277
[3381]	valid_0's binary_logloss: 0.393278
[3382]	valid_0's binary_logloss: 0.3

[3555]	valid_0's binary_logloss: 0.39322
[3556]	valid_0's binary_logloss: 0.393221
[3557]	valid_0's binary_logloss: 0.39322
[3558]	valid_0's binary_logloss: 0.39322
[3559]	valid_0's binary_logloss: 0.39322
[3560]	valid_0's binary_logloss: 0.393221
[3561]	valid_0's binary_logloss: 0.393219
[3562]	valid_0's binary_logloss: 0.39322
[3563]	valid_0's binary_logloss: 0.393219
[3564]	valid_0's binary_logloss: 0.393219
[3565]	valid_0's binary_logloss: 0.393219
[3566]	valid_0's binary_logloss: 0.39322
[3567]	valid_0's binary_logloss: 0.39322
[3568]	valid_0's binary_logloss: 0.393221
[3569]	valid_0's binary_logloss: 0.393222
[3570]	valid_0's binary_logloss: 0.393221
[3571]	valid_0's binary_logloss: 0.393222
[3572]	valid_0's binary_logloss: 0.393222
[3573]	valid_0's binary_logloss: 0.393222
[3574]	valid_0's binary_logloss: 0.393222
[3575]	valid_0's binary_logloss: 0.393222
[3576]	valid_0's binary_logloss: 0.393221
[3577]	valid_0's binary_logloss: 0.39322
[3578]	valid_0's binary_logloss: 0.393219


[3751]	valid_0's binary_logloss: 0.393169
[3752]	valid_0's binary_logloss: 0.39317
[3753]	valid_0's binary_logloss: 0.39317
[3754]	valid_0's binary_logloss: 0.393169
[3755]	valid_0's binary_logloss: 0.393169
[3756]	valid_0's binary_logloss: 0.393169
[3757]	valid_0's binary_logloss: 0.393169
[3758]	valid_0's binary_logloss: 0.393169
[3759]	valid_0's binary_logloss: 0.393169
[3760]	valid_0's binary_logloss: 0.393169
[3761]	valid_0's binary_logloss: 0.393169
[3762]	valid_0's binary_logloss: 0.393167
[3763]	valid_0's binary_logloss: 0.393166
[3764]	valid_0's binary_logloss: 0.393165
[3765]	valid_0's binary_logloss: 0.393166
[3766]	valid_0's binary_logloss: 0.393167
[3767]	valid_0's binary_logloss: 0.393167
[3768]	valid_0's binary_logloss: 0.393168
[3769]	valid_0's binary_logloss: 0.393167
[3770]	valid_0's binary_logloss: 0.393168
[3771]	valid_0's binary_logloss: 0.393168
[3772]	valid_0's binary_logloss: 0.393168
[3773]	valid_0's binary_logloss: 0.393167
[3774]	valid_0's binary_logloss: 0.3

[3947]	valid_0's binary_logloss: 0.393123
[3948]	valid_0's binary_logloss: 0.393124
[3949]	valid_0's binary_logloss: 0.393124
[3950]	valid_0's binary_logloss: 0.393122
[3951]	valid_0's binary_logloss: 0.393121
[3952]	valid_0's binary_logloss: 0.393121
[3953]	valid_0's binary_logloss: 0.39312
[3954]	valid_0's binary_logloss: 0.39312
[3955]	valid_0's binary_logloss: 0.39312
[3956]	valid_0's binary_logloss: 0.39312
[3957]	valid_0's binary_logloss: 0.393121
[3958]	valid_0's binary_logloss: 0.39312
[3959]	valid_0's binary_logloss: 0.393121
[3960]	valid_0's binary_logloss: 0.393121
[3961]	valid_0's binary_logloss: 0.393121
[3962]	valid_0's binary_logloss: 0.39312
[3963]	valid_0's binary_logloss: 0.39312
[3964]	valid_0's binary_logloss: 0.393121
[3965]	valid_0's binary_logloss: 0.39312
[3966]	valid_0's binary_logloss: 0.393119
[3967]	valid_0's binary_logloss: 0.393118
[3968]	valid_0's binary_logloss: 0.393118
[3969]	valid_0's binary_logloss: 0.393118
[3970]	valid_0's binary_logloss: 0.393119


In [10]:
print(gbm.best_score)
print(gbm.best_iteration)

defaultdict(<class 'dict'>, {'valid_0': {'binary_logloss': 0.39311371175777043}})
0


In [11]:
gbm.save_model('lightgbm_cv'+str(fold)+'_4000.txt')

# [Sec 2] predict the testing set , and submission

In [1]:
# since the memory limmit , we restart our python process
import pandas as pd
import numpy as np
import random

In [2]:
fold=5570

In [3]:
X_test = pd.read_csv("data/test")

In [4]:
sampleSubmission = pd.read_csv("data/sampleSubmission")

In [5]:
(X_test.id == sampleSubmission.id).all()

True

In [6]:
# create testing features
from sklearn.feature_extraction import FeatureHasher
hasher = FeatureHasher(n_features=10,
            non_negative=True,
            input_type='string')

# create features

## site
tmp_features = hasher.transform(X_test.site_id.tolist()).toarray()
for i in range(10):
    X_test['site_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_test.site_domain.tolist()).toarray()
for i in range(10):
    X_test['site_domain'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_test.site_category.tolist()).toarray()
for i in range(10):
    X_test['site_category'+str(i)]=tmp_features[:,i]

## app
tmp_features = hasher.transform(X_test.app_id.tolist()).toarray()
for i in range(10):
    X_test['app_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_test.app_domain.tolist()).toarray()
for i in range(10):
    X_test['app_domain'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_test.app_category.tolist()).toarray()
for i in range(10):
    X_test['app_category'+str(i)]=tmp_features[:,i]

## devices
tmp_features = hasher.transform(X_test.device_id.tolist()).toarray()
for i in range(10):
    X_test['device_id'+str(i)]=tmp_features[:,i]

tmp_features = hasher.transform(X_test.device_ip.tolist()).toarray()
for i in range(10):
    X_test['device_ip'+str(i)]=tmp_features[:,i]
    
tmp_features = hasher.transform(X_test.device_model.tolist()).toarray()
for i in range(10):
    X_test['device_model'+str(i)]=tmp_features[:,i]

In [7]:
Site_list = ['site_id'+str(i) for i in range(10)]+['site_domain'+str(i) for i in range(10)]+ \
            ['site_category'+str(i) for i in range(10)]

App_list = ['app_id'+str(i) for i in range(10)]+['app_domain'+str(i) for i in range(10)]+ \
            ['app_category'+str(i) for i in range(10)]

Devices_list = ['device_id'+str(i) for i in range(10)]+['device_ip'+str(i) for i in range(10)]+ \
            ['device_model'+str(i) for i in range(10)]
X_test_trial = X_test[['banner_pos','C1','device_type','device_conn_type','C14','C15','C16','C17','C18',\
                         'C19','C20','C21']+Site_list+App_list+Devices_list]


In [8]:
X_test_trial.shape

(4577464, 102)

In [9]:
X_train = []
X_test = []
import gc
gc.collect()

50

In [10]:
import lightgbm as lgb
#restore model
bst = lgb.Booster(model_file='lightgbm_cv'+str(fold)+'_4000.txt')


In [16]:
y_hat = bst.predict(X_test_trial,num_iteration=4000) #,num_iteration=19311

In [17]:
sampleSubmission.click = y_hat
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("naive_feature_hashing_cv"+str(fold)+"_4000.csv",index=False)

In [15]:
y_hat[:5]

array([ 0.06895633,  0.18392689,  0.2191574 ,  0.03896161,  0.29682179])